In [1]:
from scipy.io import arff
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

data, metadata = arff.loadarff(open('data/EEG-Eye-State.arff'))

eeg_data = pd.DataFrame(data)

eeg_data = eeg_data.replace('\n','',regex=True)

eeg_data['eyeDetection'] = eeg_data['eyeDetection'].str.decode("utf-8")

In [ ]:
labels = eeg_data.pop('eyeDetection')
train_labels = pd.get_dummies(labels)
train_labels = train_labels.values
print(train_labels.shape)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(eeg_data,train_labels,test_size=0.2)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
ops.reset_default_graph()

n_x, m = np.transpose(x_train).shape
n_y = np.transpose(y_train).shape[0]
print(n_y,m, n_x)
h = 1000
costs = []

X = tf.placeholder(tf.float32, [n_x, None], name="X")
Y = tf.placeholder(tf.float32, [n_y, None], name="Y")

W1 = tf.get_variable("W1", [h, n_x], initializer = tf.contrib.layers.xavier_initializer(seed=1))
b1 = tf.get_variable("b1", [h, 1], initializer = tf.zeros_initializer())
W2 = tf.get_variable("W2", [n_y, h], initializer = tf.contrib.layers.xavier_initializer(seed=1))
b2 = tf.get_variable("b2", [n_y, 1], initializer = tf.zeros_initializer())

Z1 = tf.add(tf.matmul(W1, X), b1)  
A1 = tf.nn.relu(Z1)                                   
Z2 = tf.add(tf.matmul(W2, A1), b2)                     

logits = tf.transpose(Z2)
labels = tf.transpose(Y)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

init = tf.global_variables_initializer()

In [2]:
num_epochs = 100
minibatch_size = 50
seed = 1
with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = tf.shuffle_batch(X_train, Y_train, min_after_dequeue=1000, batch_sizes = minibatch_size, seed=1)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)

NameError: name 'init' is not defined